In [ ]:
from pydantic import BaseModel, EmailStr, ValidationError, Field, model_validator, create_model, ConfigDict, AfterValidator
from typing import List, Dict, Any, Optional, Tuple, Callable, TypeVar, Type, Annotated
from datetime import date
import re
import json
from pathlib import Path
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

T = TypeVar("T")


In [ ]:

class ValidatorRegistry:
    """
    Custom class that acts as a validator repository. Used to improve modularity and encapsulation.
    """
    def __init__(self):
        # Type hint for field validator factories: (field_name, error, params) -> actual_validator_callable(value)
        self._field_validators: Dict[str, Callable[[str, str, Dict[str, Any]], Callable[[Any], Any]]] = {}
        # Type hint for model validator factories: (fields_list, error, params) -> actual_model_validator_callable(values_dict)
        self._model_validators: Dict[str, Callable[[List[str], str, Dict[str, Any]], Callable[[BaseModel, Dict[str, Any]], Dict[str, Any]]]] = {}
        
        self._field_validator_params: Dict[str, List[str]] = {}
        self._model_validator_params: Dict[str, List[str]] = {}
        self._field_validator_metadata: Dict[str, Dict[str, Any]] = {}
        self._model_validator_metadata: Dict[str, Dict[str, Any]] = {}
        # New: Dictionary to store dynamically created Pydantic schemas
        self._schemas: Dict[str, Type[BaseModel]] = {} 
        
    def register_field_validator(
        self,
        validator_type: str,
        validator_func_factory: Callable[[str, str, Dict[str, Any]], Callable[[Any], Any]], # Factory signature
        required_params: List[str] = None,
        **kwargs: Any
    ) -> None:
        if validator_type in self._field_validators:
            logger.warning(f"Field validator type '{validator_type}' already registered. Overwriting.")
        self._field_validators[validator_type] = validator_func_factory
        self._field_validator_params[validator_type] = required_params or []
        self._field_validator_metadata[validator_type] = kwargs
        logger.info(f"Registered field validator: {validator_type}")

    def get_field_validator(self, validator_type: str) -> Optional[Callable[[str, str, Dict[str, Any]], Callable[[Any], Any]]]:
        return self._field_validators.get(validator_type)

    def get_field_params(self, validator_type: str) -> List[str]:
        return self._field_validator_params.get(validator_type, [])

    def get_field_validator_types(self) -> List[str]:
        return list(self._field_validators.keys())

    def register_model_validator(
        self,
        validator_type: str,
        validator_func_factory: Callable[[List[str], str, Dict[str, Any]], Callable[[BaseModel, Dict[str, Any]], Dict[str, Any]]], # Factory signature
        required_params: List[str] = None,
        **kwargs: Any
    ) -> None:
        if validator_type in self._model_validators:
            logger.warning(f"Model validator type '{validator_type}' already registered. Overwriting.")
        self._model_validators[validator_type] = validator_func_factory
        self._model_validator_params[validator_type] = required_params or []
        self._model_validator_metadata[validator_type] = kwargs
        logger.info(f"Registered model validator: {validator_type}")

    def get_model_validator(self, validator_type: str) -> Optional[Callable[[List[str], str, Dict[str, Any]], Callable[[BaseModel, Dict[str, Any]], Dict[str, Any]]]]:
        return self._model_validators.get(validator_type)

    def get_model_params(self, validator_type: str) -> List[str]:
        return list(self._model_validator_params.get(validator_type, []))

    def get_model_validator_types(self) -> List[str]:
        return list(self._model_validators.keys())

    # New: Methods to manage dynamically created schemas
    def register_schema(self, schema_name: str, schema_model: Type[BaseModel]) -> None:
        """Registers a dynamically created Pydantic schema."""
        if schema_name in self._schemas:
            logger.warning(f"Schema '{schema_name}' already registered. Overwriting.")
        self._schemas[schema_name] = schema_model
        logger.info(f"Registered schema: {schema_name}")

    def get_schema(self, schema_name: str) -> Optional[Type[BaseModel]]:
        """Retrieves a registered Pydantic schema."""
        return self._schemas.get(schema_name)


In [97]:

class ValidatorConfig(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True) 
    registry: 'ValidatorRegistry' = Field(exclude=True) 
    type: str
    error: str
    params: Dict[str, Any] = {}

    @model_validator(mode='after')
    def check_params(self) -> 'ValidatorConfig':
        if not self.registry:
            raise ValueError("ValidatorRegistry not provided during ValidatorConfig instantiation.")
        
        if self.type not in self.registry.get_field_validator_types():
            logger.error(f"Unknown field validator type '{self.type}' during validation.")
            raise ValueError(f"Unknown field validator type '{self.type}'")
        
        required_params = self.registry.get_field_params(self.type)
        for param in required_params:
            if param not in self.params:
                logger.error(f"Missing required parameter '{param}' for validator '{self.type}'.")
                raise ValueError(f"Missing required parameter '{param}' for validator '{self.type}'")
        return self

class FieldConfig(BaseModel):
    # FieldConfig might not directly need arbitrary_types_allowed if it only contains other BaseModels
    # and primitive types, as we can  add  if any nested custom types might appear.
    model_config = ConfigDict(arbitrary_types_allowed=True) 
    name: str
    type: str
    required: bool = True
    validators: List[ValidatorConfig] = []

In [98]:
class ModelValidatorConfig(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True) # Add this line
    registry: 'ValidatorRegistry' = Field(exclude=True) 
    type: str
    fields: List[str]
    error: str
    params: Dict[str, Any] = {}

    @model_validator(mode='after')
    def check_params(self) -> 'ModelValidatorConfig':
        if not self.registry:
            raise ValueError("ValidatorRegistry not provided during ModelValidatorConfig instantiation.")

        if self.type not in self.registry.get_model_validator_types():
            logger.error(f"Unknown model validator type '{self.type}' during validation.")
            raise ValueError(f"Unknown model validator type '{self.type}'")
        
        required_params = self.registry.get_model_params(self.type)
                # ensure that when you define a validator of a certain type, you have provided all required parameters 
        for param in required_params:#it will take all the params required for that type of validator
            if param not in self.params:
                logger.error(f"Missing required parameter '{param}' for model validator '{self.type}'.")
                raise ValueError(f"Missing required parameter '{param}' for model validator '{self.type}'")
        return self


In [99]:
class SchemaConfig(BaseModel):
    # SchemaConfig also needs arbitrary_types_allowed because it contains FieldConfig and ModelValidatorConfig,
    # which now directly contain the ValidatorRegistry (even if it's excluded from dump).
    model_config = ConfigDict(arbitrary_types_allowed=True)
    schema_name: str
    fields: List[FieldConfig]
    model_validators: List[ModelValidatorConfig] = []

In [100]:

class ValidatorManager:
    """
    Primary class for managing plugin of custom validators
    Supports registration through a custom json file stored on python path
    Also runtime registration through register-*-* funnction. Currently these registrations are not persisted
    """
    def __init__(self, registry: ValidatorRegistry):
        self.registry = registry

    def register_field_validator(
        self,
        validator_type: str,
        validator_func: Callable[[str, str, Dict[str, Any]], Callable],
        required_params: List[str] = None,
        **kwargs: Any
    ) -> None:
        self.registry.register_field_validator(validator_type, validator_func, required_params, **kwargs)
        logger.info(f"Dynamically registered field validator: {validator_type}")

    def register_model_validator(
        self,
        validator_type: str,
        validator_func: Callable[[List[str], str, Dict[str, Any]], Callable],
        required_params: List[str] = None,
        **kwargs: Any
    ) -> None:
        self.registry.register_model_validator(validator_type, validator_func, required_params, **kwargs)
        logger.info(f"Dynamically registered model validator: {validator_type}")

    def load_validator_plugins(self, config_path: str = "validators.json") -> None:
        config_file = Path(config_path)
        if not config_file.exists():
            logger.info(f"Validator plugin file not found at {config_path}. Skipping plugin loading.")
            return

        try:
            with config_file.open() as f:
                config = json.load(f)
            logger.info(f"Loaded validator plugins from {config_path}.")
        except json.JSONDecodeError as e:
            logger.error(f"Invalid JSON in plugin file {config_path}: {e}")
            raise ValueError(f"Invalid JSON in plugin file {config_path}: {e}")

        for validator in config.get("field_validators", []):
            v_type = validator.get("type")
            module_name = validator.get("module")
            func_name = validator.get("function")
            required_params = validator.get("required_params", [])
            description = validator.get("description", "")

            if not all([v_type, module_name, func_name]):
                logger.error(f"Missing required fields in field validator configuration: {validator}")
                raise ValueError(f"Missing required fields in field validator: {validator}")

            try:
                module = __import__(module_name, fromlist=[func_name])
                validator_func = getattr(module, func_name)
                self.registry.register_field_validator(
                    v_type, validator_func, required_params, description=description
                )
                logger.debug(f"Successfully loaded field validator '{v_type}' from {module_name}.{func_name}")
            except (ImportError, AttributeError) as e:
                logger.error(f"Failed to load field validator '{v_type}' from {module_name}.{func_name}: {e}")
                raise ValueError(f"Failed to load field validator '{v_type}' from {module_name}.{func_name}: {e}")

        for validator in config.get("model_validators", []):
            v_type = validator.get("type")
            module_name = validator.get("module")
            func_name = validator.get("function")
            required_params = validator.get("required_params", [])
            description = validator.get("description", "")

            if not all([v_type, module_name, func_name]):
                logger.error(f"Missing required fields in model validator configuration: {validator}")
                raise ValueError(f"Missing required fields in model validator: {validator}")

            try:
                module = __import__(module_name, fromlist=[func_name]) #used to import the module from the json file
                validator_func = getattr(module, func_name)#imports the function present in that module
                self.registry.register_model_validator(
                    v_type, validator_func, required_params, description=description
                )
                logger.debug(f"Successfully loaded model validator '{v_type}' from {module_name}.{func_name}")
            except (ImportError, AttributeError) as e:
                logger.error(f"Failed to load model validator '{v_type}' from {module_name}.{func_name}: {e}")
                raise ValueError(f"Failed to load model validator '{v_type}' from {module_name}.{func_name}: {e}")


In [101]:


def setup_fields_and_validators(config: SchemaConfig, registry: Type[ValidatorRegistry]) -> Tuple[Dict[str, Any], Dict[str, List[Callable]]]:
    """
    Parses the SchemaConfig to determine Pydantic fields and
    applies standard Pydantic Field constraints, and also identifies
    any custom field validators that need to be applied.
    """
    fields = {}
    # This will store actual callables for field validators to be attached
    raw_field_validators_by_field: Dict[str, List[Callable]] = {}

    for field_name, rules in config.fields.items():
        field_type = rules.get("type")
        pydantic_field = None
        field_annotations = None
        field_constraints = {}
        custom_field_validators = [] # List to hold custom validators for this field

        # Standard Pydantic Field types and constraints
        if field_type == "string":
            field_annotations = str
            if "min_length" in rules:
                field_constraints["min_length"] = rules["min_length"]
            if "max_length" in rules:
                field_constraints["max_length"] = rules["max_length"]
            if "pattern" in rules:
                field_constraints["pattern"] = rules["pattern"]

        elif field_type == "float":
            field_annotations = float
            if "gt" in rules:
                field_constraints["gt"] = rules["gt"]
            if "ge" in rules:
                field_constraints["ge"] = rules["ge"]
            if "lt" in rules:
                field_constraints["lt"] = rules["lt"]
            if "le" in rules:
                field_constraints["le"] = rules["le"]

        elif field_type == "integer":
            field_annotations = int
            if "gt" in rules:
                field_constraints["gt"] = rules["gt"]
            if "ge" in rules:
                field_constraints["ge"] = rules["ge"]
            if "lt" in rules:
                field_constraints["lt"] = rules["lt"]
            if "le" in rules:
                field_constraints["le"] = rules["le"]

        elif field_type == "enum":
            if "values" not in rules:
                raise ValueError(f"Enum type for '{field_name}' must have 'values' specified.")
            enum_values = tuple(rules["values"])
            field_annotations = Literal.__getitem__(enum_values)

        elif field_type == "date":
            field_annotations = date

        elif field_type == "email":
            field_annotations = EmailStr

        else:
            raise ValueError(f"Unsupported field type: {field_type} for field {field_name}")

        # Instantiate Pydantic Field. If no specific constraints, make it required by default.
        pydantic_field = Field(...) if not field_constraints else Field(**field_constraints)

        # Handle custom field validators if defined in rules (e.g., {"not_empty": True})
        if "custom_validators" in rules and isinstance(rules["custom_validators"], dict):
            for validator_type, params in rules["custom_validators"].items():
                validator_info = registry.get_field_validator(validator_type)
                if validator_info:
                    validator_func_factory = validator_info["func"]
                    required_params = validator_info["required_params"]

                    # Prepare parameters for the validator factory
                    validator_params = {p: params[p] for p in required_params if p in params}
                    if not all(p in validator_params for p in required_params):
                        raise ValueError(f"Missing required parameters for custom validator '{validator_type}' on field '{field_name}'. Required: {required_params}")

                    # Create the actual validator function
                    if validator_type == "not_empty": # Special case for not_empty, which doesn't need params beyond field_name
                         custom_validator_callable = validator_func_factory(field_name)
                    else:
                        custom_validator_callable = validator_func_factory(field_name, **validator_params)

                    custom_field_validators.append(custom_validator_callable)
                else:
                    logger.warning(f"Custom field validator type '{validator_type}' for field '{field_name}' not registered.")

        fields[field_name] = (field_annotations, pydantic_field)
        if custom_field_validators:
            raw_field_validators_by_field[field_name] = custom_field_validators

    return fields, raw_field_validators_by_field


def setup_model_validators_for_dynamic_model(config: SchemaConfig, registry: Type[ValidatorRegistry]) -> List[Callable]:
    """
    Sets up model-level validators.
    For this example, let's add a custom validator to check if end_date is after start_date.
    """
    validators = []

    # Example: Add a custom model validator for date range
    if "start_date" in config.fields and "end_date" in config.fields:
        # Pydantic V2 model_validator in 'after' mode receives the Pydantic model instance
        def validate_date_range(self, data: Any) -> Any:
            # Access validated data directly from the model instance 'self'
            start_date_val = data.get("start_date")
            end_date_val = data.get("end_date")

            if start_date_val and end_date_val: # Ensure both are present and valid date objects
                if end_date_val < start_date_val:
                    raise ValueError("end_date cannot be before start_date")
            return data
        validate_date_range.__name__ = "validate_transaction_date_range"
        validators.append(validate_date_range)

    return validators


# --- Main Dynamic Schema Creation Function ---
def create_dynamic_schema(config: SchemaConfig, registry: Type[ValidatorRegistry]) -> Type[BaseModel]:
    """
    Dynamically creates a Pydantic BaseModel based on a SchemaConfig object.

    Args:
        config (SchemaConfig): The configuration object containing schema name and field rules.
        registry (ValidatorRegistry): The registry to store the created schema.

    Returns:
        Type[BaseModel]: A Pydantic BaseModel class.
    """
    logger.info(f"Creating dynamic schema for '{config.schema_name}'")

    if not config.fields:
        logger.warning(f"No 'fields' found in the configuration for schema '{config.schema_name}'. Creating an empty schema.")

    # Call helper functions
    fields, raw_field_validators_by_field = setup_fields_and_validators(config, registry)
    raw_model_validator_functions = setup_model_validators_for_dynamic_model(config, registry)

    # Create the dynamic model
    dynamic_model = create_model(
        config.schema_name,
        **fields,
        __base__=BaseModel
    )

    # Attach custom field validators using root_validator if needed for cross-field/complex logic
    # For simple field-level validators, it's often more straightforward to define them
    # directly on the Field using `PydanticCustomValidator` or `BeforeValidator`/`AfterValidator`
    # within `setup_fields_and_validators` if Pydantic 2.x's validation callables are used.
    # The `raw_field_validators_by_field` would be used to create `PydanticCustomValidator`
    # or similar constructs and passed as the second element of the tuple in `fields`.

    # Dynamically attach model validators
    for i, model_val_func in enumerate(raw_model_validator_functions):
        # Attach the model_validator decorated function to the dynamic model
        method_name = f'_dynamic_model_validator_{i}_{model_val_func.__name__}'
        # This applies the @model_validator decorator to the function and then assigns it.
        setattr(dynamic_model, method_name, model_validator(mode="after")(model_val_func))
        logger.debug(f"Attached dynamic model validator '{model_val_func.__name__}' as '{method_name}' to schema.")

    logger.info(f"Dynamic schema '{config.schema_name}' created successfully.")
    registry.register_schema(config.schema_name, dynamic_model) # Register the created schema
    return dynamic_model


In [102]:

# Individual field validator functions (factories that create the actual validator functions)
def create_not_empty_validator(field_name: str, error: str, params: Dict[str, Any]) -> Callable:
    def not_empty_validator(cls, v):
        if not v:
            logger.debug(f"Validation failed for '{field_name}': {error}")
            raise ValueError(error)
        return v
    return not_empty_validator

def create_length_validator(field_name: str, error: str, params: Dict[str, Any]) -> Callable:
    length = params["length"]
    def length_validator(cls, v):
        if len(v) != length:
            logger.debug(f"Validation failed for '{field_name}' (length {len(v)} != {length}): {error}")
            raise ValueError(error)
        return v
    return length_validator

def create_regex_validator(field_name: str, error: str, params: Dict[str, Any]) -> Callable:
    pattern = params["pattern"]
    def regex_validator(cls, v):
        if not re.match(pattern, str(v)):
            logger.debug(f"Validation failed for '{field_name}' (value '{v}' no match regex '{pattern}'): {error}")
            raise ValueError(error)
        return v
    return regex_validator

def create_enum_validator(field_name: str, error: str, params: Dict[str, Any]) -> Callable:
    values = params["values"]
    def enum_validator(cls, v):
        if v not in values:
            logger.debug(f"Validation failed for '{field_name}' (value '{v}' not in enum {values}): {error}")
            raise ValueError(error)
        return v
    return enum_validator

def create_date_after_validator(field_name: str, error: str, params: Dict[str, Any]) -> Callable:
    min_date = date.fromisoformat(params["min_date"])
    def date_after_validator(cls, v: date):
        if v < min_date:
            logger.debug(f"Validation failed for '{field_name}' (date {v} < {min_date}): {error}")
            raise ValueError(error)
        return v
    return date_after_validator

def create_min_value_validator(field_name: str, error: str, params: Dict[str, Any]) -> Callable:
    min_val = params["min_val"]
    def min_value(cls, v):
        if v < min_val:
            logger.debug(f"Validation failed for '{field_name}' (value {v} < {min_val}): {error}")
            raise ValueError(error)
        return v
    return min_value

def create_max_value_validator(field_name: str, error: str, params: Dict[str, Any]) -> Callable:
    max_val = params["max_val"]
    def max_value(cls, v):
        if v > max_val:
            logger.debug(f"Validation failed for '{field_name}' (value {v} > {max_val}): {error}")
            raise ValueError(error)
        return v
    return max_value

def create_no_na_validator(field_name: str, error: str, params: Dict[str, Any]) -> Callable:
    invalid_values = {"N/A", "NA", "null", "NULL", "", None}
    def no_na_validator(cls, v):
        if v in invalid_values:
            logger.debug(f"Validation failed for '{field_name}' (value '{v}' is an invalid 'NA' value): {error}")
            raise ValueError(error)
        return v
    return no_na_validator

def create_length_range_validator(field_name: str, error: str, params: Dict[str, Any]) -> Callable:
    min_length = params.get("min_length", 0)
    max_length = params.get("max_length", float('inf'))
    def length_range_validator(cls, v):
        if not (min_length <= len(v) <= max_length):
            logger.debug(f"Validation failed for '{field_name}' (length {len(v)} not in range [{min_length}, {max_length}]): {error}")
            raise ValueError(error)
        return v
    return length_range_validator

def create_number_range_validator(field_name: str, error: str, params: Dict[str, Any]) -> Callable:
    min_value = params.get("min_value", float('-inf'))
    max_value = params.get("max_value", float('inf'))
    def number_range_validator(cls, v):
        if not (min_value <= v <= max_value):
            logger.debug(f"Validation failed for '{field_name}' (value {v} not in range [{min_value}, {max_value}]): {error}")
            raise ValueError(error)
        return v
    return number_range_validator

def create_date_range_validator(field_name: str, error: str, params: Dict[str, Any]) -> Callable:
    min_date = date.fromisoformat(params.get("min_date", "1900-01-01"))
    max_date = date.fromisoformat(params.get("max_date", "2100-12-31"))
    def date_range_validator(cls, v: date):
        if not (min_date <= v <= max_date):
            logger.debug(f"Validation failed for '{field_name}' (date {v} not in range [{min_date}, {max_date}]): {error}")
            raise ValueError(error)
        return v
    return date_range_validator

# Model validators
def create_model_date_order_validator(fields: List[str], error: str, params: Dict[str, Any]) -> Callable:
    def date_order_validator(cls: Any, values: Dict[str, Any]) -> Dict[str, Any]:
        date1 = values.get(fields[0])
        date2 = values.get(fields[1])

        if date1 is None or date2 is None:
            return values

        if not (isinstance(date1, date) and isinstance(date2, date)):
            logger.error(f"Model validation error: 'date_order' expected date types for fields {fields}, got {type(date1).__name__}, {type(date2).__name__}")
            raise TypeError(f"Invalid types for date order validation. Expected 'date'.")

        if date2 < date1:
            logger.debug(f"Model validation failed (date order): {fields[1]} ({date2}) is before {fields[0]} ({date1})")
            raise ValueError(error)
        return values
    return date_order_validator

def create_model_sum_equals_validator(fields: List[str], error: str, params: Dict[str, Any]) -> Callable:
    target_sum = params["target_sum"]
    def sum_equals_validator(cls: Any, values: Dict[str, Any]) -> Dict[str, Any]:
        field_values = []
        for field_name in fields:
            val = values.get(field_name)
            if not isinstance(val, (int, float)):
                logger.error(f"Model validation error: 'sum_equals' expected numeric types for fields {fields}, got {type(val).__name__} for {field_name}")
                raise TypeError(f"Invalid types for sum equals validation. Expected 'int' or 'float'.")
            field_values.append(val)
            
        current_sum = sum(field_values)

        if current_sum != target_sum:
            logger.debug(f"Model validation failed (sum equals): {current_sum} != {target_sum}")
            raise ValueError(error)
        return values
    return sum_equals_validator


def setup_fields_and_validators(config: SchemaConfig, registry: ValidatorRegistry) -> (Dict[str, Any], Dict[str, List[Callable]]):
    """
    Parses the SchemaConfig to determine Pydantic fields and
    sets up field-level validators if any custom ones were to be defined.
    (Currently, this only handles the standard Pydantic Field definitions).
    """
    fields = {}
    raw_field_validators_by_field = {} # Not used in this basic example, but kept for signature

    for field_name, rules in config.fields.items():
        field_type = rules.get("type")
        pydantic_field = None
        field_annotations = None

        if field_type == "string":
            field_annotations = str
            field_constraints = {}
            if "min_length" in rules:
                field_constraints["min_length"] = rules["min_length"]
            if "max_length" in rules:
                field_constraints["max_length"] = rules["max_length"]
            if "pattern" in rules:
                field_constraints["pattern"] = rules["pattern"]
            pydantic_field = Field(...) if field_constraints else Field(...) # Use Field(...) for required
            if field_constraints:
                 pydantic_field = Field(**field_constraints)


        elif field_type == "float":
            field_annotations = float
            field_constraints = {}
            if "gt" in rules:
                field_constraints["gt"] = rules["gt"]
            if "ge" in rules:
                field_constraints["ge"] = rules["ge"]
            if "lt" in rules:
                field_constraints["lt"] = rules["lt"]
            if "le" in rules:
                field_constraints["le"] = rules["le"]
            pydantic_field = Field(...) if field_constraints else Field(...)
            if field_constraints:
                pydantic_field = Field(**field_constraints)

        elif field_type == "integer":
            field_annotations = int
            field_constraints = {}
            if "gt" in rules:
                field_constraints["gt"] = rules["gt"]
            if "ge" in rules:
                field_constraints["ge"] = rules["ge"]
            if "lt" in rules:
                field_constraints["lt"] = rules["lt"]
            if "le" in rules:
                field_constraints["le"] = rules["le"]
            pydantic_field = Field(...) if field_constraints else Field(...)
            if field_constraints:
                pydantic_field = Field(**field_constraints)

        elif field_type == "enum":
            if "values" not in rules:
                raise ValueError(f"Enum type for '{field_name}' must have 'values' specified.")
            enum_values = tuple(rules["values"])
            field_annotations = Literal.__getitem__(enum_values)
            pydantic_field = Field(...)

        elif field_type == "date":
            field_annotations = date
            pydantic_field = Field(...)

        elif field_type == "email":
            field_annotations = EmailStr
            pydantic_field = Field(...)

        else:
            raise ValueError(f"Unsupported field type: {field_type} for field {field_name}")

        fields[field_name] = (field_annotations, pydantic_field)

    return fields, raw_field_validators_by_field


def setup_model_validators_for_dynamic_model(config: SchemaConfig, registry: ValidatorRegistry) -> List[Callable]:
    """
    Sets up model-level validators.
    For this example, let's add a custom validator to check if end_date is after start_date.
    """
    validators = []

    # Example: Add a custom model validator for date range
    if "start_date" in config.fields and "end_date" in config.fields:
        def validate_date_range(data: Any) -> Any:
            if data.get("start_date") and data.get("end_date"):
                # Pydantic 2.x handles dates as date objects, so comparison works directly
                if data["end_date"] < data["start_date"]:
                    raise ValueError("end_date cannot be before start_date")
            return data
        # Give the validator a name for clarity in logs/errors
        validate_date_range.__name__ = "validate_transaction_date_range"
        validators.append(validate_date_range)

    return validators


# --- Main Dynamic Schema Creation Function ---
def create_dynamic_schema(config: SchemaConfig, registry: ValidatorRegistry) -> Type[BaseModel]:
    """
    Dynamically creates a Pydantic BaseModel based on a SchemaConfig object.

    Args:
        config (SchemaConfig): The configuration object containing schema name and field rules.
        registry (ValidatorRegistry): The registry to store the created schema.

    Returns:
        Type[BaseModel]: A Pydantic BaseModel class.
    """
    logger.info(f"Creating dynamic schema for '{config.schema_name}'")

    if not config.fields:
        logger.warning(f"No 'fields' found in the configuration for schema '{config.schema_name}'. Creating an empty schema.")

    # Pass the config object directly, no need to read from file inside this function
    fields, raw_field_validators_by_field = setup_fields_and_validators(config, registry)
    raw_model_validator_functions = setup_model_validators_for_dynamic_model(config, registry)

    pydantic_field_validators = {}
    for field_name, validators_list in raw_field_validators_by_field.items():
        # Using a tuple of (validator, mode) for Pydantic v2
        pydantic_field_validators[field_name] = [(v, 'after') for v in validators_list]

    # Create the dynamic model
    dynamic_model = create_model(
        config.schema_name,
        **fields,
        # In Pydantic V2, field validators are typically passed directly in Field()
        # The __validators__ argument is primarily for model_validator.
        # However, for backward compatibility or complex dynamic cases, it might be used.
        # For simplicity with the current setup, we'll just rely on Field() constraints.
        __base__=BaseModel
    )

    # Dynamically attach model validators
    for i, model_val_func in enumerate(raw_model_validator_functions):
        # We need to assign the decorated function to the class itself.
        # The model_validator decorator returns a callable that can be assigned.
        setattr(dynamic_model, f'_dynamic_model_validator_{i}', model_validator(mode="after")(model_val_func))
        logger.debug(f"Attached dynamic model validator '{model_val_func.__name__}' to schema.")

    logger.info(f"Dynamic schema '{config.schema_name}' created successfully.")
    registry.register_schema(config.schema_name, dynamic_model) # Register the created schema
    return dynamic_model


#
# Initialize the registry and manager
validator_registry = ValidatorRegistry()
validator_manager = ValidatorManager(validator_registry)

# Register all field validators
validator_manager.register_field_validator("not_empty", create_not_empty_validator, required_params=[], description="Ensures field is not empty")
validator_manager.register_field_validator("length", create_length_validator, required_params=["length"], description="Validates exact string length")
validator_manager.register_field_validator("regex", create_regex_validator, required_params=["pattern"], description="Validates against regex pattern")
validator_manager.register_field_validator("enum", create_enum_validator, required_params=["values"], description="Ensures value is in allowed set")
validator_manager.register_field_validator("date_after", create_date_after_validator, required_params=["min_date"], description="Validates date after minimum")
validator_manager.register_field_validator("min_value", create_min_value_validator, required_params=["min_val"], description="Ensures value meets minimum")
validator_manager.register_field_validator("max_value", create_max_value_validator, required_params=["max_val"], description="Ensures value meets maximum")
validator_manager.register_field_validator("no_na", create_no_na_validator, required_params=[], description="Prevents 'NA' values")
validator_manager.register_field_validator("length_range", create_length_range_validator, required_params=["min_length", "max_length"], description="Validates string length within a range")
validator_manager.register_field_validator("number_range", create_number_range_validator, required_params=["min_value", "max_value"], description="Ensures numeric value is within a range")
validator_manager.register_field_validator("date_range", create_date_range_validator, required_params=["min_date", "max_date"], description="Ensures date is within a specific range")

# Register all model validators
validator_manager.register_model_validator("date_order", create_model_date_order_validator, required_params=[], description="Ensures second date is after first")
validator_manager.register_model_validator("sum_equals", create_model_sum_equals_validator, required_params=["target_sum"], description="Validates sum of fields")


2025-06-02 03:39:30,451 - __main__ - INFO - Registered field validator: not_empty
2025-06-02 03:39:30,452 - __main__ - INFO - Dynamically registered field validator: not_empty
2025-06-02 03:39:30,452 - __main__ - INFO - Registered field validator: length
2025-06-02 03:39:30,452 - __main__ - INFO - Dynamically registered field validator: length
2025-06-02 03:39:30,452 - __main__ - INFO - Registered field validator: regex
2025-06-02 03:39:30,453 - __main__ - INFO - Dynamically registered field validator: regex
2025-06-02 03:39:30,453 - __main__ - INFO - Registered field validator: enum
2025-06-02 03:39:30,453 - __main__ - INFO - Dynamically registered field validator: enum
2025-06-02 03:39:30,453 - __main__ - INFO - Registered field validator: date_after
2025-06-02 03:39:30,454 - __main__ - INFO - Dynamically registered field validator: date_after
2025-06-02 03:39:30,454 - __main__ - INFO - Registered field validator: min_value
2025-06-02 03:39:30,454 - __main__ - INFO - Dynamically regi

In [103]:

# Type mapping for dynamic schema creation
TYPE_MAPPING = {
    "str": str,
    "int": int,
    "float": float,
    "date": date,
    "EmailStr": EmailStr
}

def setup_fields_and_validators(config: SchemaConfig, registry: ValidatorRegistry) -> Tuple[Dict[str, Any], Dict[str, Any]]:
    fields = {}
    raw_field_validators_by_field: Dict[str, List[Callable]] = {} 
    
    for field_config in config.fields:
        field_name = field_config.name
        
        field_type = TYPE_MAPPING.get(field_config.type)
        if field_type is None:
            logger.error(f"Unsupported field type '{field_config.type}' for field '{field_name}'.")
            raise ValueError(f"Unsupported field type '{field_config.type}' for field '{field_name}'.")

        is_required = field_config.required
        
        fields[field_name] = (field_type, ...) if is_required else (Optional[field_type], None)
        logger.debug(f"Setting up field '{field_name}' with type '{field_config.type}', required: {is_required}")
        
        raw_field_validators_by_field[field_name] = []
        for validator_config in field_config.validators:
            validator_func_factory = registry.get_field_validator(validator_config.type)
            if not validator_func_factory:
                logger.error(f"No field validator function factory registered for type '{validator_config.type}' during application.")
                raise ValueError(f"No field validator function factory registered for type '{validator_config.type}'")
            
            raw_validator_func = validator_func_factory(
                field_name, validator_config.error, validator_config.params
            )
            raw_field_validators_by_field[field_name].append(raw_validator_func)
            logger.debug(f"Prepared raw field validator '{validator_config.type}' for field '{field_name}'")
            
    return fields, raw_field_validators_by_field

def setup_model_validators_for_dynamic_model(config: SchemaConfig, registry: ValidatorRegistry) -> List[Callable]:
    model_validator_methods = []
    for model_validator_config in config.model_validators:
        validator_func_factory = registry.get_model_validator(model_validator_config.type)
        if not validator_func_factory:
            logger.error(f"No model validator function factory registered for type '{model_validator_config.type}' during application.")
            raise ValueError(f"No model validator function factory registered for type '{model_validator_config.type}'")
        
        model_validator_methods.append(
            validator_func_factory(
                fields=model_validator_config.fields,
                error=model_validator_config.error,
                params=model_validator_config.params
            )
        )
        logger.debug(f"Prepared model validator '{model_validator_config.type}' for fields {model_validator_config.fields}")
    return model_validator_methods




# --- Main Dynamic Schema Creation Function ---
def create_dynamic_schema(config: SchemaConfig, registry: ValidatorRegistry) -> Type[BaseModel]:
    """
    Dynamically creates a Pydantic BaseModel based on a SchemaConfig object.

    Args:
        config (SchemaConfig): The configuration object containing schema name and field rules.
        registry (ValidatorRegistry): The registry to store the created schema.

    Returns:
        Type[BaseModel]: A Pydantic BaseModel class.
    """
    logger.info(f"Creating dynamic schema for '{config.schema_name}'")

    if not config.fields:
        logger.warning(f"No 'fields' found in the configuration for schema '{config.schema_name}'. Creating an empty schema.")

    fields, raw_field_validators_by_field = setup_fields_and_validators(config, registry)
    raw_model_validator_functions = setup_model_validators_for_dynamic_model(config, registry)

    # In Pydantic V2, __validators__ is typically used for field validators that are not directly
    # defined in Field() or for more advanced cases. Model validators are attached differently.
    # For simplicity, we are handling field constraints directly in setup_fields_and_validators
    # and model validators via setattr below.
    dynamic_model = create_model(
        config.schema_name,
        **fields,
        __base__=BaseModel
    )

    # Dynamically attach model validators using setattr
    for i, model_val_func in enumerate(raw_model_validator_functions):
        # The model_validator decorator needs to be applied to a method of the class.
        # We're dynamically adding methods to the created model.
        # model_validator(mode="after")(model_val_func) makes 'model_val_func' a class method
        # and applies the Pydantic model_validator logic.
        method_name = f'_dynamic_model_validator_{i}_{model_val_func.__name__}'
        # Apply the decorator and then set the attribute
        setattr(dynamic_model, method_name, model_validator(mode="after")(model_val_func))
        logger.debug(f"Attached dynamic model validator '{model_val_func.__name__}' as '{method_name}' to schema.")

    logger.info(f"Dynamic schema '{config.schema_name}' created successfully.")
    registry.register_schema(config.schema_name, dynamic_model) # Register the created schema
    return dynamic_model

# Validation Functions
def validate_transaction(data: Dict[str, Any], schema: Type[BaseModel]) -> Dict[str, Any]:
    logger.info(f"\n--- Validating Data ---")
    logger.info(f"Input data: {data}")
    logger.info(f"Original types in input: {{k: type(v).__name__ for k, v in data.items()}}")
    try:
        # No need to pass context here for the data validation, as registry is
        # now passed directly to ValidatorConfig and ModelValidatorConfig objects
        validated = schema.model_validate(data) 
        validated_dump = validated.model_dump()
        logger.info(f"Validated output: {validated_dump}")
        logger.info(f"Coerced types in output: {{k: type(v).__name__ for k, v in validated_dump.items()}}")
        logger.info(f"Validation successful.")
        return validated_dump
    except ValidationError as e:
        logger.warning(f"Validation failed for data: {data}. Errors: {e.errors()}")
        return {"errors": e.errors()}

def process_data(rows: List[Dict[str, Any]], schema: Type[BaseModel]) -> List[Dict[str, Any]]:
    logger.info(f"Processing {len(rows)} data rows.")
    return [validate_transaction(row, schema) for row in rows]


In [104]:
# Pass the validator_registry to each ValidatorConfig and ModelValidatorConfig instance
schema_config_data = {
    "schema_name": "TransactionSchema",
    "fields": [
        {
            "name": "Transaction_ID",
            "type": "str",
            "required": True,
            "validators": [
                {"type": "not_empty", "error": "ID cannot be null", "params": {}},
                {"type": "length", "error": "ID must be exactly 8 characters", "params": {"length": 8}},
                {"type": "regex", "error": "ID must start with a letter", "params": {"pattern": r"^[A-Za-z][A-Za-z0-9]{7}$"}},
                {"type": "no_na", "error": "ID cannot be 'NA'", "params": {}},
            ],
        },
        {
            "name": "Email",
            "type": "EmailStr",
            "required": True,
            "validators": [
                {"type": "regex", "error": "Invalid email format", "params": {"pattern": r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[A-Za-z]{2,}$"}},
                {"type": "no_na", "error": "Email cannot be 'NA'", "params": {}},
            ],
        },
        {
            "name": "Gender",
            "type": "str",
            "required": True,
            "validators": [
                {"type": "not_empty", "error": "Gender cannot be empty", "params": {}},
                {"type": "enum", "error": "Gender must be 'Male', 'Female', or 'Other'", "params": {"values": ["Male", "Female", "Other"]}},
            ],
        },
        {
            "name": "Date",
            "type": "date",
            "required": True,
            "validators": [
                {"type": "date_after", "error": "Date cannot be before 2023-01-01", "params": {"min_date": "2023-01-01"}},
                {"type": "date_range", "error": "Date must be within 2023 and 2025", "params": {"min_date": "2023-01-01", "max_date": "2025-12-31"}},
            ],
        },
        {
            "name": "firstpurchase",
            "type": "date",
            "required": True,
            "validators": [
                {"type": "date_after", "error": "First purchase date cannot be before 2023-01-01", "params": {"min_date": "2023-01-01"}},
            ],
        },
        {
            "name": "lastpurchase",
            "type": "date",
            "required": False,
            "validators": [
                {"type": "date_after", "error": "Last purchase date cannot be before 2023-01-01", "params": {"min_date": "2023-01-01"}},
            ],
        },
        {
            "name": "Amount",
            "type": "float",
            "required": True,
            "validators": [
                {"type": "min_value", "error": "Amount must be positive", "params": {"min_val": 0.01}},
                {"type": "number_range", "error": "Amount must be between 0.01 and 9999.99", "params": {"min_value": 0.01, "max_value": 9999.99}},
            ],
        },
        {
            "name": "Tax",
            "type": "float",
            "required": True,
            "validators": [
                {"type": "min_value", "error": "Tax cannot be negative", "params": {"min_val": 0.0}},
                {"type": "max_value", "error": "Tax cannot exceed 50.0", "params": {"max_val": 50.0}},
            ],
        },
        {
            "name": "Feedback",
            "type": "str",
            "required": False,
            "validators": [
                {"type": "length_range", "error": "Feedback must be between 10 and 500 characters if provided", "params": {"min_length": 10, "max_length": 500}},
            ],
        },
    ],
    "model_validators": [
        {
            "type": "date_order",
            "fields": ["firstpurchase", "lastpurchase"],
            "error": "Last purchase must be after first purchase",
        },
        {
            "type": "sum_equals",
            "fields": ["Amount", "Tax"],
            "error": "Amount and Tax must sum to 100",
            "params": {"target_sum": 100.0},
        },
    ],
}


In [105]:
# Manually inject the registry into each relevant config object
for field in schema_config_data["fields"]:
    for validator in field["validators"]:
        validator['registry'] = validator_registry

for model_validator_cfg in schema_config_data["model_validators"]:
    model_validator_cfg['registry'] = validator_registry

schema_config = SchemaConfig.model_validate(schema_config_data)

DynamicTransactionSchema = create_dynamic_schema(schema_config, validator_registry)


2025-06-02 03:39:30,471 - __main__ - INFO - Creating dynamic schema for 'TransactionSchema'
2025-06-02 03:39:30,473 - __main__ - INFO - Dynamic schema 'TransactionSchema' created successfully.
2025-06-02 03:39:30,473 - __main__ - INFO - Registered schema: TransactionSchema


In [106]:
# Example Usage:
valid_data = {
    "Transaction_ID": "ABC12345",
    "Email": "test@example.com",
    "Gender": "Male",
    "Date": "2024-05-20",
    "firstpurchase": "2024-01-01",
    "lastpurchase": "2024-02-15",
    "Amount": 90.0,
    "Tax": 10.0,
    "Feedback": "This is a great product, very satisfied with the service."
}

invalid_data = {
    "Transaction_ID": "A123",  # Too short
    "Email": "invalid-email",
    "Gender": "Unknown",
    "Date": "2022-01-01",  # Too early
    "firstpurchase": "2024-03-01",
    "lastpurchase": "2024-02-01",  # Last purchase before first
    "Amount": -5.0,  # Negative amount
    "Tax": 60.0,  # Too high
    "Feedback": "Too short."  # Too short feedback
}

sum_error_data = {
    "Transaction_ID": "DEF67890",
    "Email": "another@example.com",
    "Gender": "Female",
    "Date": "2024-06-10",
    "firstpurchase": "2024-01-01",
    "lastpurchase": "2024-03-15",
    "Amount": 50.0,
    "Tax": 20.0,
    "Feedback": "Everything was good, but the shipping was slow."
}

print("\n--- Validating Valid Data ---")
validate_transaction(valid_data, DynamicTransactionSchema)

print("\n--- Validating Invalid Data ---")
validate_transaction(invalid_data, DynamicTransactionSchema)

print("\n--- Validating Data with Sum Error ---")
validate_transaction(sum_error_data, DynamicTransactionSchema)

print("\n--- Validating Data with Optional Field Missing ---")
optional_data = {
    "Transaction_ID": "GHI12345",
    "Email": "optional@example.com",
    "Gender": "Other",
    "Date": "2024-07-01",
    "firstpurchase": "2024-01-01",
    "Amount": 70.0,
    "Tax": 30.0,
}
validate_transaction(optional_data, DynamicTransactionSchema)

print("\n--- Validating Data with Optional Field Present but Empty/Invalid ---")
optional_invalid_data = {
    "Transaction_ID": "JKL67890",
    "Email": "empty@example.com",
    "Gender": "Male",
    "Date": "2024-08-01",
    "firstpurchase": "2024-01-01",
    "lastpurchase": "2024-03-15",
    "Amount": 80.0,
    "Tax": 20.0,
    "Feedback": ""
}

validate_transaction(optional_invalid_data, DynamicTransactionSchema)


2025-06-02 03:39:30,477 - __main__ - INFO - 
--- Validating Data ---
2025-06-02 03:39:30,477 - __main__ - INFO - Input data: {'Transaction_ID': 'ABC12345', 'Email': 'test@example.com', 'Gender': 'Male', 'Date': '2024-05-20', 'firstpurchase': '2024-01-01', 'lastpurchase': '2024-02-15', 'Amount': 90.0, 'Tax': 10.0, 'Feedback': 'This is a great product, very satisfied with the service.'}
2025-06-02 03:39:30,477 - __main__ - INFO - Original types in input: {k: type(v).__name__ for k, v in data.items()}
2025-06-02 03:39:30,478 - __main__ - INFO - Validated output: {'Transaction_ID': 'ABC12345', 'Email': 'test@example.com', 'Gender': 'Male', 'Date': datetime.date(2024, 5, 20), 'firstpurchase': datetime.date(2024, 1, 1), 'lastpurchase': datetime.date(2024, 2, 15), 'Amount': 90.0, 'Tax': 10.0, 'Feedback': 'This is a great product, very satisfied with the service.'}



--- Validating Valid Data ---


2025-06-02 03:39:30,478 - __main__ - INFO - Coerced types in output: {k: type(v).__name__ for k, v in validated_dump.items()}
2025-06-02 03:39:30,478 - __main__ - INFO - Validation successful.
2025-06-02 03:39:30,478 - __main__ - INFO - 
--- Validating Data ---
2025-06-02 03:39:30,478 - __main__ - INFO - Input data: {'Transaction_ID': 'A123', 'Email': 'invalid-email', 'Gender': 'Unknown', 'Date': '2022-01-01', 'firstpurchase': '2024-03-01', 'lastpurchase': '2024-02-01', 'Amount': -5.0, 'Tax': 60.0, 'Feedback': 'Too short.'}
2025-06-02 03:39:30,479 - __main__ - INFO - Original types in input: {k: type(v).__name__ for k, v in data.items()}
2025-06-02 03:39:30,479 - __main__ - WARNING - Validation failed for data: {'Transaction_ID': 'A123', 'Email': 'invalid-email', 'Gender': 'Unknown', 'Date': '2022-01-01', 'firstpurchase': '2024-03-01', 'lastpurchase': '2024-02-01', 'Amount': -5.0, 'Tax': 60.0, 'Feedback': 'Too short.'}. Errors: [{'type': 'value_error', 'loc': ('Email',), 'msg': 'value 


--- Validating Invalid Data ---

--- Validating Data with Sum Error ---

--- Validating Data with Optional Field Missing ---

--- Validating Data with Optional Field Present but Empty/Invalid ---


{'Transaction_ID': 'JKL67890',
 'Email': 'empty@example.com',
 'Gender': 'Male',
 'Date': datetime.date(2024, 8, 1),
 'firstpurchase': datetime.date(2024, 1, 1),
 'lastpurchase': datetime.date(2024, 3, 15),
 'Amount': 80.0,
 'Tax': 20.0,
 'Feedback': ''}

In [107]:
sample_data = [
    {
        "Transaction_ID": "A1234567",
        "customer_name": "Alice Smith",
        "amount": 50.50,
        "currency": "USD",
        "start_date": "2023-03-15",
        "end_date": "2023-03-20",
        "status": "Completed",
        "item_count": 49,
        "customer_email": "alice@example.com"
    },
    {
        "Transaction_ID": "B9876543",
        "customer_name": "Bob Johnson",
        "amount": 120.00,
        "currency": "EUR",
        "start_date": "2024-01-10",
        "end_date": "2023-12-01", # Intentionally wrong end_date for test
        "status": "Pending",
        "item_count": 0, # Should fail min_value for item_count
        "customer_email": "bob@example.com"
    },
    {
        "Transaction_ID": "C1234567", # Invalid regex
        "customer_name": "", # Not empty
        "amount": -10.00, # Min value
        "currency": "YEN", # Enum
        "start_date": "2022-01-01", # Date after
        "end_date": "2023-02-01",
        "status": "NA", # No NA
        "item_count": 50,
        "customer_email": "charlie@invalid" # Invalid email
    },
    {
        "Transaction_ID": "A0000001",
        "customer_name": "Valid Customer With Very Very Very Very Very Very Very Long Name That Exceeds The Limit",
        "amount": 75.00,
        "currency": "GBP",
        "start_date": "2024-05-01",
        "end_date": "2024-05-10",
        "status": "Processing",
        "item_count": 25,
        "customer_email": "valid@example.com"
    }
]

# --- Main part of your script ---

try:


    # Create the dynamic schema
    DynamicTransactionSchema = create_dynamic_schema(schema_config, validator_registry)

    print(f"Schema fields: {list(DynamicTransactionSchema.model_fields.keys())}")

    # Validate sample data
    for i, data in enumerate(sample_data):
        print(f"\n--- Validating Sample Data {i+1} ---")
        logger.info(f"Input data: {data}")
        logger.info(f"Original types in input: {{k: type(v).__name__ for k, v in data.items()}}")
        try:
            validated_data = DynamicTransactionSchema.model_validate(data)
            print("Validation Successful:")
            print(json.dumps(validated_data.model_dump(), indent=2, default=str))
        except ValidationError as e:
            logger.warning(f"Validation failed for data: {data}. Errors: {e.errors()}")
            print("Validation Failed:")
            print(e.json(indent=2))

except Exception as e:
    logger.error(f"An unexpected error occurred during schema creation or validation: {e}", exc_info=True)


2025-06-02 03:39:30,498 - __main__ - INFO - Creating dynamic schema for 'TransactionSchema'
2025-06-02 03:39:30,501 - __main__ - INFO - Dynamic schema 'TransactionSchema' created successfully.
2025-06-02 03:39:30,501 - __main__ - WARNING - Schema 'TransactionSchema' already registered. Overwriting.
2025-06-02 03:39:30,501 - __main__ - INFO - Registered schema: TransactionSchema
2025-06-02 03:39:30,501 - __main__ - INFO - Input data: {'Transaction_ID': 'A1234567', 'customer_name': 'Alice Smith', 'amount': 50.5, 'currency': 'USD', 'start_date': '2023-03-15', 'end_date': '2023-03-20', 'status': 'Completed', 'item_count': 49, 'customer_email': 'alice@example.com'}
2025-06-02 03:39:30,501 - __main__ - INFO - Original types in input: {k: type(v).__name__ for k, v in data.items()}
2025-06-02 03:39:30,502 - __main__ - WARNING - Validation failed for data: {'Transaction_ID': 'A1234567', 'customer_name': 'Alice Smith', 'amount': 50.5, 'currency': 'USD', 'start_date': '2023-03-15', 'end_date': '2

Schema fields: ['Transaction_ID', 'Email', 'Gender', 'Date', 'firstpurchase', 'lastpurchase', 'Amount', 'Tax', 'Feedback']

--- Validating Sample Data 1 ---
Validation Failed:
[
  {
    "type": "missing",
    "loc": [
      "Email"
    ],
    "msg": "Field required",
    "input": {
      "Transaction_ID": "A1234567",
      "customer_name": "Alice Smith",
      "amount": 50.5,
      "currency": "USD",
      "start_date": "2023-03-15",
      "end_date": "2023-03-20",
      "status": "Completed",
      "item_count": 49,
      "customer_email": "alice@example.com"
    },
    "url": "https://errors.pydantic.dev/2.11/v/missing"
  },
  {
    "type": "missing",
    "loc": [
      "Gender"
    ],
    "msg": "Field required",
    "input": {
      "Transaction_ID": "A1234567",
      "customer_name": "Alice Smith",
      "amount": 50.5,
      "currency": "USD",
      "start_date": "2023-03-15",
      "end_date": "2023-03-20",
      "status": "Completed",
      "item_count": 49,
      "customer_ema